# PDF table exptraction  

In [ ]:
import os
import sys
import tabula
import PyPDF2
import os
import pandas as pd

In [ ]:
path = os.getcwd()
path = path+'/PDFs/'


### Reading pdf files using tabula and pypdf

In [ ]:

# reading PDF file to extract tables from
file = path+"./B.pdf"
tables = tabula.read_pdf(file, multiple_tables=True, pages='all')
pdf_file = PyPDF2.PdfFileReader(open(file,'rb'))

In [ ]:
# get number of pages in pdf file 
totalPages = pdf_file.numPages
print(totalPages)


### First attempt to find and correct 90 deg rotated pages 

In [ ]:
for i in range(1,totalPages):
    #print (pdf_file.getPage(i).get('/Rotate'))
    #check if file is rotated at any angle, if yes, print the information in the file
    if (pdf_file.getPage(i).get('/Rotate') != None and (pdf_file.getPage(i).get('/Rotate')) != 0):
        print("Page: "+str(i)+" "+"Angle: "+str(pdf_file.getPage(i).get('/Rotate')))
        

## cleaning tables 
1. drop based on shape - smaler than 2 in row or column will be droped
2. drop if number of NAN elements are higher than others
3. drop if all columns are objects

In [ ]:
table_clean = []
i=0
j = len(tables)
while i < j:
    #print(i)
    
    if (tables[i-1].shape[1]<2) or (tables[i-1].shape[0]<2):
        #del tables[i-1]
        #j-=1
        i+=1
        continue
    if tables[i-1].isna().sum().sum() > tables[i-1].count().sum():
        
        i+=1
        continue
    
  
    df = pd.DataFrame(tables[(i-1)])
    
    for column in df.columns:
        if (df[column].dtypes != 'object') and (df[column].sum() > df[column].isna().sum()):
            drop = False
            break
        else:
            drop = True
    #print("drop: ", drop)
    if drop:
        #del tables[i-1]
        #j-=1
        i+=1
        continue
    table_clean.append(df)
    i+=1
#print(i)


In [ ]:
table_clean[8]

In [ ]:
table_clean[5]["Unnamed: 1"].isna().sum()

In [ ]:
info = pdfReader.getDocumentInfo()

In [ ]:
info

In [ ]:
page

In [ ]:
tables[10].df

# 